In [50]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import gym
import numpy as np

In [42]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
print(f'Using: {str(device).upper()}')

Using: CUDA


In [43]:
# Define the model
class RLModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RLModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [46]:
# Create an instance of the RL model
input_size, hidden_size, output_size = 4, 16, 2
model = RLModel(input_size, hidden_size, output_size).to(device)

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters())

# Load the environment
env = gym.make("CartPole-v1")

In [ ]:
# Train the model
num_episodes, discount_rate = 500, 0.95
for episode in range(num_episodes):
    state = env.reset()
    done = False
    while not done:
        state = torch.tensor(np.array(state)).float().to(device)
        action_values = model(state)
        action = torch.argmax(action_values).item()
        next_state, reward, done, _ = env.step(action)
        next_state = torch.tensor(next_state).float().to(device)
        target = reward + discount_rate * torch.max(model(next_state))
        action_values[action] = target
        loss = criterion(action_values, model(state))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        state = next_state

print("Finished Training")